In [36]:
import requests
import os
from dotenv import find_dotenv, load_dotenv
from langchain.chains import LLMChain
from langchain_google_genai import GoogleGenerativeAI
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

load_dotenv(find_dotenv())
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
HUGGINGFACE_API_TOKEN = os.getenv("HUGGINGFACE_API_TOKEN")
AUDIO_FILE_NAME = r"speech_nb.wav"

In [37]:
def text2story():
    llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)
    return LLMChain(
        llm=llm,
        prompt=PromptTemplate.from_template(
            "Tell a story no longer than 50 words, based on this context: {context}"
        ),
        output_parser=StrOutputParser(),
    )


def text2audio(text):
    API_URL = "https://api-inference.huggingface.co/models/facebook/mms-tts-eng"
    headers = {"Authorization": f"Bearer {HUGGINGFACE_API_TOKEN}"}
    audio_bytes = requests.post(API_URL, headers=headers, json={"inputs": text}).content
    with open(AUDIO_FILE_NAME, mode="wb") as f:
        f.write(audio_bytes)
    return


def execute_chain(text):
    story = text2story().run(text)
    text2audio(story)
    return story, AUDIO_FILE_NAME

In [38]:
import gradio as gr

demo = gr.Interface(
    fn=execute_chain,
    inputs=gr.Textbox(label="Story context", lines=3, placeholder="Enter here"),
    outputs=[
        gr.Textbox(label="Full story"),
        gr.Audio(label="Story audio"),
    ],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
